In [66]:
import pandas as pd

df = pd.read_excel('Data/online_retail_II.xlsx', sheet_name='Year 2010-2011')

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [67]:
df=df[(df["Quantity"]>0)]

df['Total Price'] = df["Quantity"] * df["Price"]
customer_df = df.groupby('Customer ID').agg({
    'Customer ID': 'count',
    'Quantity': 'sum',
    'Total Price' : 'sum',
})

customer_df.columns=['total_transaction','total_unit','total_price']

customer_df['avg_price'] = customer_df['total_price'] / customer_df['total_transaction']

customer_df.head()

,total_transaction,total_unit,total_price,avg_price
Customer ID,,,,
12346.0,1,74215,77183.60,77183.60
12347.0,182,2458,4310.00,784420.00
12348.0,31,2341,1797.24,55714.44
12349.0,73,631,1757.55,128301.15
12350.0,17,197,334.40,5684.80


In [68]:
customer_df['frequency'] = customer_df['total_transaction'] / customer_df.shape[0]

repeat_rate = len(customer_df[customer_df['total_transaction'] > 2]) / customer_df.shape[0]

print(repeat_rate)
customer_df.head()

0.9711915187831297


,total_transaction,total_unit,total_price,avg_price,frequency
Customer ID,,,,,
12346.0,1,74215,77183.60,77183.60,0.000230
12347.0,182,2458,4310.00,784420.00,0.041945
12348.0,31,2341,1797.24,55714.44,0.007145
12349.0,73,631,1757.55,128301.15,0.016824
12350.0,17,197,334.40,5684.80,0.003918


In [71]:
churn_rate = 1 - repeat_rate

customer_df['cltv'] = ((customer_df['avg_price'] * customer_df['frequency']) / churn_rate *  customer_df["total_price"] * 0.05).astype(float).round(2)

In [73]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(1, 100))

customer_df['Scaled_cltv'] = scaler.fit_transform(customer_df[['cltv']])

def getsegment(num):
    if num >= 75:
        return "A"
    elif num >= 50:
        return "B"
    elif num >= 25:
        return "C"
    else:
        return "D"

customer_df['segment'] = customer_df['Scaled_cltv'].apply(getsegment)

# Display the result
print(customer_df.head())

             total_transaction  total_unit  total_price     avg_price  \
Customer ID                                                             
14911.0                   5677       80515    143825.06  8.164949e+08   
14646.0                   2080      197491    280206.02  5.828285e+08   
14096.0                   5111       16352     65164.79  3.330572e+08   
17841.0                   7847       23071     40991.57  3.216608e+08   
14156.0                   1400       57885    117379.63  1.643315e+08   

             frequency          cltv  Scaled_cltv segment  
Customer ID                                                
14911.0       1.308366  2.666655e+14   100.000000       A  
14646.0       0.479373  1.358756e+14    49.394128       C  
14096.0       1.177921  4.437085e+13    13.988649       D  
17841.0       1.808481  4.138628e+13    12.833845       D  
14156.0       0.322655  1.080193e+13     1.000000       D  
